## Data


In [1]:
import json
with open('../data/teczno/GeoJSON-file/North-Carolina-2014.geojson') as f:
    data = json.load(f)
data = data['features']

In [2]:
data[0]['properties']

{'COUNTY_ID': 11,
 'COUNTY_NAM': 'BUNCOMBE',
 'ENR_DESC': '09.1_MURPHY-OAKLEY COMMUNITY CENTER',
 'GID': 1,
 'OF_PREC_ID': None,
 'PREC_ID': '09.1',
 'area_km2': 7,
 'asian_pop': 18,
 'black_pop': 994,
 'college_pop': 1539,
 'con_blue_votes': 897,
 'con_contested': 'True',
 'con_districts': 10,
 'con_red_votes': 386,
 'county_name': 'BUNCOMBE',
 'diploma_pop': 892,
 'education_pop': 3085,
 'graduate_pop': 304,
 'hispanic_pop': 192,
 'median_age': 40,
 'median_income': 34526,
 'other_pop': 115,
 'population': 4298,
 'precinct_id': '09.1',
 'school_pop': 351,
 'sen_blue': 899,
 'sen_red': 331,
 'sldl_blue_votes': 1008,
 'sldl_contested': 'No',
 'sldl_districts': 114,
 'sldl_red_votes': None,
 'sldu_blue_votes': 916,
 'sldu_contested': 'Yes',
 'sldu_districts': 49,
 'sldu_red_votes': 367,
 'white_pop': 2979}

## Efficiency Gap

In [3]:
def votes_cast_for_senator(pt):
    red  = pt['properties']['sen_red']
    blue = pt['properties']['sen_blue']
    return (red, blue)

In [4]:
def threshold(votes):
    if votes[0] == votes[1]:
        raise Exception("The Efficiency Gap is undefined in the case of a tie")
    half = sum(votes)/2
    if round(half) == half:
        thresh = (half) + 1
    else:
        thresh = round(half)
    return int(thresh)

In [5]:
def wasted_votes(pt):
    (red, blue) = votes_cast_for_senator(pt)
    if red==blue:
        return (0,0)
    
    thresh = threshold((red, blue))
    if red>thresh:
        red_wasted = red - thresh
        blue_wasted = blue
    else:
        blue_wasted = blue - thresh
        red_wasted = red
    
    return (red_wasted, blue_wasted)

In [6]:
for pt in data:
    (red, blue) = votes_cast_for_senator(pt)
    thresh = threshold((red, blue))
    (red_wasted, blue_wasted) = wasted_votes(pt)
    
    print(f"With {red} red votes and {blue} blue votes, {thresh} votes are needed to win.")
    print(f"Red wasted {red_wasted} votes and blue wasted {blue_wasted} votes.\n")

With 331 red votes and 899 blue votes, 616 votes are needed to win.
Red wasted 331 votes and blue wasted 283 votes.

With 614 red votes and 473 blue votes, 544 votes are needed to win.
Red wasted 70 votes and blue wasted 473 votes.

With 132 red votes and 652 blue votes, 393 votes are needed to win.
Red wasted 132 votes and blue wasted 259 votes.

With 73 red votes and 839 blue votes, 457 votes are needed to win.
Red wasted 73 votes and blue wasted 382 votes.

With 726 red votes and 464 blue votes, 596 votes are needed to win.
Red wasted 130 votes and blue wasted 464 votes.

With 19 red votes and 324 blue votes, 172 votes are needed to win.
Red wasted 19 votes and blue wasted 152 votes.

With 231 red votes and 466 blue votes, 348 votes are needed to win.
Red wasted 231 votes and blue wasted 118 votes.

With 358 red votes and 309 blue votes, 334 votes are needed to win.
Red wasted 24 votes and blue wasted 309 votes.

With 423 red votes and 432 blue votes, 428 votes are needed to win.
Re

Exception: The Efficiency Gap is undefined in the case of a tie

In [ ]:
def net_wasted_votes(data):
    total_red_wasted = 0
    total_blue_wasted = 0
    for ii, pt in enumerate(data):
        (red_wasted, blue_wasted) = wasted_votes(pt)
        total_red_wasted  += red_wasted
        total_blue_wasted += blue_wasted
    return total_red_wasted - total_blue_wasted

In [ ]:
def total_votes_cast(data):
    total = 0
    for pt in data:
        total += sum(votes_cast_for_senator(pt))
    return total

In [ ]:
def efficiency_gap(data):
    return net_wasted_votes(data)/total_votes_cast(data)

In [ ]:
print('For all of North Carolina:')
print('Net Wasted Votes: ', net_wasted_votes(data))
gap = efficiency_gap(data)
print('Efficiency Gap: ', str(round(gap*100, 2)) + '%')
print('This fucked over the {}.\n'.format('Democrats' if gap<0 else 'Republicans'))

for district in range(1,14):
    precincts = [pt for pt in data if district==pt['properties']['con_districts']]
    print('District: ', district)
    print('Net Wasted Votes: ', net_wasted_votes(precincts))
    gap = efficiency_gap(precincts)
    print('Efficiency Gap: ', str(round(gap*100, 2)) + '%')
    print('This screwed over the {}.\n'.format('Democrats' if gap<0 else 'Republicans'))

#### The above method calculates the partial net wasted votes and sums them up. Does this calculate the same efficiency gap when summing total wasted votes?

In [ ]:
def total_wasted_votes(district):
    red = sum([pt['properties']['sen_red'] for pt in district])
    blue= sum([pt['properties']['sen_blue'] for pt in district])
    if red==blue:
        return (0,0)
    
    thresh = threshold((red, blue))
    if red>thresh:
        red_wasted = red - thresh
        blue_wasted = blue
    else:
        blue_wasted = blue - thresh
        red_wasted = red
    
    return (red_wasted, blue_wasted, red+blue)

In [ ]:
def efficiency_gap_from_total(data):
    (red_wasted, blue_wasted, total_votes) = total_wasted_votes(precincts)
    net_wasted = red_wasted-blue_wasted
    return (net_wasted, net_wasted/total_votes)

In [ ]:
dem_gap = 0
rep_gap = 0
dem_dist = 0
rep_dist = 0
for district in range(1,14):
    precincts = [pt for pt in data if district==pt['properties']['con_districts']]
    (net_wasted, gap) = efficiency_gap_from_total(precincts)
    print('District: ', district)
    print('Net Wasted Votes: ', net_wasted)
    print('Efficiency Gap: ', str(round(gap*100, 2)) + '%')
    print('This screwed over the {}.\n'.format('Democrats' if gap<0 else 'Republicans'))
    
    if gap<0:
        dem_gap += gap
        dem_dist += 1
    else:
        rep_gap += gap
        rep_dist += 1

dem_gap = dem_gap/dem_dist
rep_gap = rep_gap/rep_dist
print('Average Efficiency Gap for Democrats in North Carolina: ', str(round(dem_gap*100, 2)) + '%')
print('Average Efficiency Gap for Republicans in North Carolina: ', str(round(rep_gap*100, 2)) + '%')

## Recalculate to double check

In [9]:
import sys
path = '/Users/Joe/Documents/Metis/Projects/metis-05-kojak/python-scripts/'
sys.path.append(path)
from kojak import threshold as thr
from kojak import wasted_votes as wv

In [17]:
# Find wasted votes per district:
rep_wasted = []
dem_wasted = []
total_votes = 0
for district in range(1, 14):
    if district != 9:
        precincts = [pt for pt in data if pt['properties']['con_districts'] == district]
        rep_votes = 0
        dem_votes = 0
        for pt in precincts:
            rep_votes += pt['properties']['con_red_votes']
            dem_votes += pt['properties']['con_blue_votes']
            
        total_votes += rep_votes + dem_votes
        
        thresh = thr(rep_votes + dem_votes)
        rep_wasted.append(wv(rep_votes, thresh))
        dem_wasted.append(wv(dem_votes, thresh))

In [18]:
rep_wasted

[38147,
 16693,
 24216,
 40967,
 23720,
 21821,
 25469,
 25172,
 21566,
 26812,
 77692,
 16752]

In [19]:
dem_wasted

[27894,
 61134,
 44139,
 32040,
 79497,
 95330,
 72564,
 55099,
 64892,
 74010,
 84839,
 71158]

In [20]:
total_votes

2243258

In [21]:
gap = (sum(rep_wasted) - sum(dem_wasted))/total_votes

In [22]:
gap

-0.1799030695533015